In [1]:
import albumentations as Aug
from pytorch_lightning import Trainer


from careamics import (
    CAREamicsModule,
    CAREamicsTrainDataModule,
)
from careamics.transforms import ManipulateN2V


/Users/joran.deschamps/miniconda3/envs/careamics/lib/python3.8/site-packages/bioimageio/spec/shared/_resolve_source.py:437: CacheWarning: found cached /var/folders/g8/92r7j4_x0tvg2hn1rl_pd3f40000gp/T/joran.deschamps/bioimageio_cache/https/raw.githubusercontent.com/bioimage-io/bioimage.io/main/site.config.json. Skipping download of https://raw.githubusercontent.com/bioimage-io/bioimage.io/main/site.config.json.
  warnings.warn(f"found cached {local_path}. Skipping download of {uri}.", category=CacheWarning)
/Users/joran.deschamps/miniconda3/envs/careamics/lib/python3.8/site-packages/bioimageio/spec/shared/_resolve_source.py:437: CacheWarning: found cached /var/folders/g8/92r7j4_x0tvg2hn1rl_pd3f40000gp/T/joran.deschamps/bioimageio_cache/https/bioimage-io.github.io/collection-bioimage-io/collection.json. Skipping download of https://bioimage-io.github.io/collection-bioimage-io/collection.json.
  warnings.warn(f"found cached {local_path}. Skipping download of {uri}.", category=CacheWarning

In [7]:
# Instantiate ligthning module
model = CAREamicsModule(
    algorithm="n2v",
    loss="n2v", 
    architecture="UNet",
    model_parameters={
        # parameters such as depth, n2v2, etc. See UNet definition.
    },
    optimizer="Adam", # see SupportedOptimizer
    optimizer_parameters={
        "lr": 1e-4,
        # parameters from torch.optim
    },
    lr_scheduler="ReduceLROnPlateau", # see SupportedLRScheduler
    lr_scheduler_parameters={
        # parameters from torch.optim.lr_scheduler
    }
)

In [8]:
# Create trainer
trainer = Trainer(max_epochs=50)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/joran.deschamps/miniconda3/envs/careamics/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


### Possibility 1: Use CAREamics data module

In [ ]:
# define function to read data
def read_my_data_type(file):
    pass

# Create your transforms using albumentations
transforms = Aug.Compose(
    [Aug.Flip(), Aug.RandomRotate90(), Aug.Normalize(), ManipulateN2V()],
)

# Instantiate data module
train_path = ...
val_path = ...

train_data_module = CAREamicsTrainDataModule(
    train_path=train_path,
    val_path=val_path,
    data_type="custom", # this forces read_source_func to be specified
    patch_size=(64, 64),
    axes="SYX",
    batch_size=128,
    transforms=transforms,
    num_workers=4,
    read_source_func = read_my_data_type # function to read data
)

### Possibility 2: Pass your own Dataset

In [ ]:
# Write youn own lightning data module
train_data_module = ...

### Train and predict using Lightning

In [ ]:
# train model
trainer.fit(model, datamodule=train_data_module)

In [ ]:
# predict
path_to_pred_data = ...
pred_dataloader = ...

pred = trainer.predict(model, pred_dataloader)